In [2]:
# %%capture
# !pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer

In [92]:
import os
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import date
import datetime as dt
import pandas as pd
import sys

In [2]:
import psycopg2
import pandas.io.sql as sqlio
# from tabulate import tabulate

In [3]:
import json
import argparse
import lxml
import config

# from Authentication import *
import requests
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


In [13]:
url = "https://aact.ctti-clinicaltrials.org/pipe_files"
response = requests.get(url)

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text)
# print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" rel="stylesheet"/>
  <script crossorigin="anonymous" integrity="sha384-q8i/X+965DzO0rT7abK41JStQIAqVgRVzpbzo5smXKp4YfRvH+8abtTE1Pi6jizo" src="https://code.jquery.com/jquery-3.3.1.slim.min.js">
  </script>
  <script crossorigin="anonymous" integrity="sha384-IQsoLXl5PILFhosVNubq5LC7Qb9DXgDA9i+tQ8Zj3iwWAwPtgFTxbJ8NT4GN1R8p" src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.9.2/dist/umd/popper.min.js">
  </script>
  <script crossorigin="anonymous" integrity="sha384-cVKIPhGWiC2Al4u+LWgxfKTRIcfu0JTxR+EQDz/bgldoEyl4H0zUF0QKbrJ0EcQF" src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/js/bootstrap.min.js">
  </script>
  <link href="https://cdn.jsdelivr.net/npm/flatpickr/dist/flatpickr.min.css" rel="stylesheet"/>
  <script src="https

In [6]:

# https://aact.ctti-clinicaltrials.org/static/exported_files/monthly/20220301_pipe-delimited-export.zip
# https://aact.ctti-clinicaltrials.org/static/exported_files/monthly/20220801_pipe-delimited-export.zip
upload_dates = []
zip_files = []
links = []
body = soup.find_all('td', attrs={'class': 'file-archive'}) #Find all
for el in body:
    tags = el.find('a')
    try:
        if 'href' in tags.attrs:   # looking for href inside anchor tag    
            link = "https://aact.ctti-clinicaltrials.org" + tags.get('href')
            links.append(link)
            last_upload = link.split("/")[-1]
            zip_files.append(last_upload)
            date_upload = last_upload.split("_")[0]
            upload_dates.append(date_upload)    # appending link to list of links
    except:    # pass if list missing anchor tag or anchor tag does not has a href params 
        pass
    
print(upload_dates)

upload_dates = [dt.datetime.strptime(date, '%Y%m%d').date() for date in upload_dates] # convert all strings in list into datetime objects


['20220427', '20220426', '20220425', '20220424', '20220423', '20220422', '20220421', '20220420', '20220419', '20220418', '20220415', '20220414', '20220413', '20220412', '20220411', '20220410', '20220409', '20220408', '20220407', '20220406', '20220405', '20220404', '20220403', '20220402', '20220401', '20220301', '20220201', '20220104', '20211201', '20211101', '20211004', '20210901', '20210601', '20210501', '20210401', '20210301', '20210201', '20210101', '20201201', '20201101', '20201001', '20200901', '20200801', '20200701', '20200601', '20200501', '20200401', '20200301', '20200201', '20200101', '20191201', '20191101', '20191001', '20190901', '20190801', '20190701', '20190601', '20190501', '20190401', '20190301', '20190201', '20190101', '20181201', '20181101', '20181001', '20180901', '20180801', '20180701', '20180601', '20180501', '20180401', '20180314', '20180201', '20180111', '20171217', '20171102', '20171017', '20170906', '20170811', '20170703', '20170613', '20170502', '20170403', '20

In [13]:
most_recent_date = max(upload_dates)
# print(most_recent_date)

today = dt.date.today()

# UNCOMMENT LINE BELOW WHEN IN PRODUCTION, THIS IS FOR TESTING PURPOSES
most_recent_date = min(upload_dates)

# fix this to run entire script should the current date be 
if most_recent_date < today:
#     date_file_df = pd.DataFrame(list(zip(upload_dates, zip_files, links)))
#     print(links[0])
    resp = urlopen(links[0])
else:
    print("Local instance of clinical trials data is already up to date.")



2022-04-26


In [10]:
pd.set_option('display.max_colwidth', 3000)
# date_file_df[2]

In [8]:
os.getcwd()


'/Volumes/TOSHIBA EXT/ISB/clin_trial_data'

In [4]:
# connect to DB and get the column names of the table
con = None
con = psycopg2.connect(database="aact")
con.rollback()
cursor = con.cursor()

con.autocommit = True # SQL statement is treated as a transaction and is automatically committed right after it is executed
# grab the conditions
sql = '''SELECT * FROM ctgov.conditions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
conditions_df = pd.DataFrame(tuples, columns=column_names)

#grab the browse_conditions
sql = '''SELECT * FROM ctgov.browse_conditions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
browse_conditions_df = pd.DataFrame(tuples, columns=column_names)

#grab the interventions
sql = '''SELECT * FROM ctgov.interventions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
interventions_df = pd.DataFrame(tuples, columns=column_names)

#grab the browse_interventions
sql = '''SELECT * FROM ctgov.browse_interventions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
browse_interventions_df = pd.DataFrame(tuples, columns=column_names)

con.close()


In [5]:
interventions_df = interventions_df.rename(columns={'id': 'int_id',
                                                    'nct_id': 'int_nctid',
                                                    'intervention_type': 'int_type',
                                                    'name': 'int_name',
                                                    'description': 'int_description'})
interventions_df

,int_id,int_nctid,int_type,int_name,int_description
0,4547568,NCT01340339,Device,Phototherapy,Fluorescent reverse phototherapy (7 white ligh...
1,4547569,NCT01340339,Device,Phototherapy,super LED reverse phototherapy (17 bulbs arran...
2,4547570,NCT01340365,Behavioral,Tai Chi Exercise,Practicing Tai Chi exercise 4 times a week for...
3,4547571,NCT01340365,Behavioral,Tai Chi,"Usual care, individuals attend testing session..."
4,4547572,NCT01340391,Device,Omnicast,dorsal splint 2-5 weeks
...,...,...,...,...,...
685271,4139681,NCT03104712,Other,Rice + Pesto,50g available carbohydrate
685272,4139682,NCT03104465,Behavioral,Mindfulness Training,interactive. web-based mindfulness training co...
685273,4139683,NCT03104361,Biological,Platelet-Rich Plasma,Patients who meet all eligible requirements fo...
685274,4139684,NCT03104283,Drug,Apatinib,take apatinib orally until disease progression...


In [6]:
interventions_df = interventions_df.drop(columns=['int_id', 'int_description'])

In [7]:
conditions_df = conditions_df.rename(columns={'id': 'con_id',
                                              'nct_id': 'con_nctid',
                                              'name': 'con_name',
                                              'downcase_name': 'con_downcase_name'})
conditions_df

,con_id,con_nctid,con_name,con_downcase_name
0,3072834,NCT03254264,Autism Spectrum Disorder,autism spectrum disorder
1,3072835,NCT03254329,Human Milk Nutrient Reference Values,human milk nutrient reference values
2,3349485,NCT03460652,ADHD,adhd
3,3349486,NCT03460899,Diabetes Mellitus With Hypoglycemia,diabetes mellitus with hypoglycemia
4,3072836,NCT03254342,Major Depressive Disorder,major depressive disorder
...,...,...,...,...
688783,3072052,NCT03261687,Pregnancy Related,pregnancy related
688784,3072053,NCT03261687,Pelvic Girdle Pain,pelvic girdle pain
688785,3072054,NCT03261622,Faecal Incontinence,faecal incontinence
688786,3072055,NCT03261622,Fecal Incontinence,fecal incontinence


In [8]:
conditions_df = conditions_df.drop(columns=['con_id', 'con_name'])

In [9]:
browse_interventions_df = browse_interventions_df.rename(columns={'id': 'browseint_id',
                                                                  'nct_id': 'browseint_nctid',
                                                                  'mesh_term': 'browseint_meshterm',
                                                                  'downcase_mesh_term': 'browseint_meshterm_downcase',
                                                                  'mesh_type': 'browseint_meshtype'})
browse_interventions_df

,browseint_id,browseint_nctid,browseint_meshterm,browseint_meshterm_downcase,browseint_meshtype
0,4935968,NCT04327843,Haloperidol,haloperidol,mesh-list
1,4935969,NCT04327843,Haloperidol decanoate,haloperidol decanoate,mesh-list
2,4935970,NCT04327843,Antiemetics,antiemetics,mesh-ancestor
3,5198960,NCT00808223,Alefacept,alefacept,mesh-list
4,5198961,NCT00808223,Dermatologic Agents,dermatologic agents,mesh-ancestor
...,...,...,...,...,...
1335262,4400668,NCT03100786,"Antineoplastic Agents, Hormonal","antineoplastic agents, hormonal",mesh-ancestor
1335263,4400669,NCT03100786,Antineoplastic Agents,antineoplastic agents,mesh-ancestor
1335264,4400670,NCT03100565,Lidocaine,lidocaine,mesh-list
1335265,4400671,NCT03100565,"Anesthetics, Local","anesthetics, local",mesh-ancestor


In [10]:
browse_interventions_df = browse_interventions_df.drop(columns=['browseint_id', 'browseint_meshterm'])

In [11]:
browse_conditions_df = browse_conditions_df.rename(columns={'id': 'browsecon_id',
                                                            'nct_id': 'browsecon_nctid',
                                                            'mesh_term': 'browsecon_meshterm',
                                                            'downcase_mesh_term': 'browsecon_meshterm_downcase',
                                                            'mesh_type': 'browsecon_meshtype'})
browse_conditions_df

,browsecon_id,browsecon_nctid,browsecon_meshterm,browsecon_meshterm_downcase,browsecon_meshtype
0,9631397,NCT04035564,Hyponatremia,hyponatremia,mesh-list
1,9631398,NCT04035564,Water-Electrolyte Imbalance,water-electrolyte imbalance,mesh-ancestor
2,9631399,NCT04035564,Metabolic Diseases,metabolic diseases,mesh-ancestor
3,9631400,NCT04032652,Colitis,colitis,mesh-list
4,9631401,NCT04032652,"Colitis, Ulcerative","colitis, ulcerative",mesh-list
...,...,...,...,...,...
2573556,8601072,NCT03213899,Disease Attributes,disease attributes,mesh-ancestor
2573557,8601073,NCT03213899,Pathologic Processes,pathologic processes,mesh-ancestor
2573558,8601074,NCT03214016,Hypertension,hypertension,mesh-list
2573559,8601075,NCT03214016,Vascular Diseases,vascular diseases,mesh-ancestor


In [12]:
browse_conditions_df = browse_conditions_df.drop(columns=['browsecon_id', 'browsecon_meshterm'])

In [13]:
print(len(interventions_df['int_name'].unique()))
print(len(conditions_df['con_downcase_name'].unique()))
print(len(browse_conditions_df['browsecon_meshterm_downcase'].unique()))
print(len(browse_interventions_df['browseint_meshterm_downcase'].unique()))

350361
95250
4280
3744


In [14]:
conditions_df


,con_nctid,con_downcase_name
0,NCT03254264,autism spectrum disorder
1,NCT03254329,human milk nutrient reference values
2,NCT03460652,adhd
3,NCT03460899,diabetes mellitus with hypoglycemia
4,NCT03254342,major depressive disorder
...,...,...
688783,NCT03261687,pregnancy related
688784,NCT03261687,pelvic girdle pain
688785,NCT03261622,faecal incontinence
688786,NCT03261622,fecal incontinence


In [15]:
df = pd.merge(conditions_df, interventions_df, left_on='con_nctid', right_on = 'int_nctid')

In [16]:
df.shape

(1256248, 5)

In [17]:
df_dedup = df.drop_duplicates(subset = ['con_downcase_name', 'int_name'],
                                      keep = 'first').reset_index(drop = True)

In [18]:
df_dedup.shape

(972685, 5)

In [19]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_dedup.head(1000))

,con_nctid,con_downcase_name,int_nctid,int_type,int_name
0,NCT03254264,autism spectrum disorder,NCT03254264,Behavioral,Early Start Denver Model
1,NCT03254264,autism spectrum disorder,NCT03254264,Behavioral,'as-usual' intervention
2,NCT03254329,human milk nutrient reference values,NCT03254329,Other,Assessment of human milk nutrient composition
3,NCT03460652,adhd,NCT03460652,Drug,KP415 oral capsule
4,NCT03460899,diabetes mellitus with hypoglycemia,NCT03460899,Other,Euglycaemic Clamp
5,NCT03460899,diabetes mellitus with hypoglycemia,NCT03460899,Other,Hyperinsulinaemic/Hypoglycaemic Clamp
6,NCT03460899,"diabetes mellitus, type 2",NCT03460899,Other,Euglycaemic Clamp
7,NCT03460899,"diabetes mellitus, type 2",NCT03460899,Other,Hyperinsulinaemic/Hypoglycaemic Clamp
8,NCT03460899,hypoglycemia,NCT03460899,Other,Euglycaemic Clamp
9,NCT03460899,hypoglycemia,NCT03460899,Other,Hyperinsulinaemic/Hypoglycaemic Clamp


In [20]:
df_dedup[['con_downcase_name', 'int_name']].to_csv('ClinTrials_KG_edges.csv', sep ='\t', index=False)


# notes on creating edge file

go to /proj/arivale/gglusman/KGs

once there look into wellness/source/wellness_kg_edges_v1.6.tsv

notice the first line: subject	predicate	object	relation	subject_name	object_name	category	N	Type_of_relationship	Strength_of_relationship	qualifiers	qualifier_value	Bonferroni_pval

subject, predicate, object are required (maybe some others, don’t remember)

…and they should be CURIEs

once we map concepts to biolink, mesh etc we’ll have curies

meanwhile, the content you have fits subject_name, object_name

the predicate is what we made up… let’s say ‘evaluated_for’

for short

we’ll have to find a suitable biolink predicate, or suggest a new one

so interventions would be subjects, and conditions would be objects

looking at the several first lines in your file, I see some that make sense, others that seem to suggest there are data problems

for example, “metastatic lung cancer	Pathways” -> Pathways doesn’t seem to be an intervention

‘placebo’ we’ll have to suppress

make sure to include the NCT id in a (non-required) column

it’s essentially part of the provenance

we’ll want to add multiple additional columns with things like FDA approval status, sample size, etc etc etc

look also into EHR/source/ehr_risk_kg_edges_2021_05_07.tsv

…and of course replace ‘edges’ -> ‘nodes’ for their cognate nodes files

wellness/source/wellness_kg_edges_v1.6.tsv -> wellness/source/wellness_kg_nodes_v1.6.tsv

drop "control"

‘control’ is clear, ‘control group’ too… but ‘Control Ostomy Barrier’ is likely not a ‘control’


In [21]:
# first get only relevant columns from DB
ct_extract = pd.DataFrame(df_dedup[['con_nctid', 'con_downcase_name', 'int_type', 'int_name']])
ct_extract = ct_extract.rename(columns={'con_nctid': 'nctid'})
# get CURIE column for nct_id column (https://bioregistry.io/registry/clinicaltrials)

ct_extract['nctid_curie'] = ct_extract['nctid']
ct_extract['nctid_curie'] = 'clinicaltrials:' + ct_extract['nctid'].astype(str)

In [22]:
ct_extract

,nctid,con_downcase_name,int_type,int_name,nctid_curie
0,NCT03254264,autism spectrum disorder,Behavioral,Early Start Denver Model,clinicaltrials:NCT03254264
1,NCT03254264,autism spectrum disorder,Behavioral,'as-usual' intervention,clinicaltrials:NCT03254264
2,NCT03254329,human milk nutrient reference values,Other,Assessment of human milk nutrient composition,clinicaltrials:NCT03254329
3,NCT03460652,adhd,Drug,KP415 oral capsule,clinicaltrials:NCT03460652
4,NCT03460899,diabetes mellitus with hypoglycemia,Other,Euglycaemic Clamp,clinicaltrials:NCT03460899
...,...,...,...,...,...
972680,NCT03261687,pelvic girdle pain,Other,water based exercise,clinicaltrials:NCT03261687
972681,NCT03261687,pelvic girdle pain,Other,land based exercise,clinicaltrials:NCT03261687
972682,NCT03261622,faecal incontinence,Device,Sacral nerve stimulation at different stimulat...,clinicaltrials:NCT03261622
972683,NCT03261622,fecal incontinence,Device,Sacral nerve stimulation at different stimulat...,clinicaltrials:NCT03261622


In [ ]:
# let's map all the conditions to BTE
# NOTE: we are attempting to match to BTE terms, but many of the conditions terms are not standardized and are misspelled
# First map as many terms as possible to BTE. Whatever is left over, will use other ontologies to create CURIEs


In [ ]:
# recommedation to first format the output as close to sample as possible

In [23]:
final_df = pd.DataFrame(columns=['subject','predicate','object','relation','subject_name','object_name','category'])

In [24]:
# no_col_names_df.columns = col_names_df.columns
# chosen predicate = "related_to", bc "associated_with" implies a statistical relationship 
final_df['subject'] = 'condition:' + ct_extract['con_downcase_name'].astype(str)
final_df['predicate'] = 'biolink:related_to'
final_df['object'] = 'intervention:' + ct_extract['int_name'].astype(str)   # this will not all be RxNorm CURIEs since some interventions are not drugs
final_df['relation'] = 'RO:????'
final_df.subject_name = ct_extract.con_downcase_name
final_df.object_name = ct_extract.int_name
final_df.category = 'biolink:Association'
final_df['nctid'] = ct_extract['nctid']
final_df['nctid_curie'] = ct_extract['nctid_curie']


In [25]:
final_df

,subject,predicate,object,relation,subject_name,object_name,category,nctid,nctid_curie
0,condition:autism spectrum disorder,biolink:related_to,intervention:Early Start Denver Model,RO:????,autism spectrum disorder,Early Start Denver Model,biolink:Association,NCT03254264,clinicaltrials:NCT03254264
1,condition:autism spectrum disorder,biolink:related_to,intervention:'as-usual' intervention,RO:????,autism spectrum disorder,'as-usual' intervention,biolink:Association,NCT03254264,clinicaltrials:NCT03254264
2,condition:human milk nutrient reference values,biolink:related_to,intervention:Assessment of human milk nutrient...,RO:????,human milk nutrient reference values,Assessment of human milk nutrient composition,biolink:Association,NCT03254329,clinicaltrials:NCT03254329
3,condition:adhd,biolink:related_to,intervention:KP415 oral capsule,RO:????,adhd,KP415 oral capsule,biolink:Association,NCT03460652,clinicaltrials:NCT03460652
4,condition:diabetes mellitus with hypoglycemia,biolink:related_to,intervention:Euglycaemic Clamp,RO:????,diabetes mellitus with hypoglycemia,Euglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
...,...,...,...,...,...,...,...,...,...
972680,condition:pelvic girdle pain,biolink:related_to,intervention:water based exercise,RO:????,pelvic girdle pain,water based exercise,biolink:Association,NCT03261687,clinicaltrials:NCT03261687
972681,condition:pelvic girdle pain,biolink:related_to,intervention:land based exercise,RO:????,pelvic girdle pain,land based exercise,biolink:Association,NCT03261687,clinicaltrials:NCT03261687
972682,condition:faecal incontinence,biolink:related_to,intervention:Sacral nerve stimulation at diffe...,RO:????,faecal incontinence,Sacral nerve stimulation at different stimulat...,biolink:Association,NCT03261622,clinicaltrials:NCT03261622
972683,condition:fecal incontinence,biolink:related_to,intervention:Sacral nerve stimulation at diffe...,RO:????,fecal incontinence,Sacral nerve stimulation at different stimulat...,biolink:Association,NCT03261622,clinicaltrials:NCT03261622


In [73]:
final_df.to_csv('ClinTrials_KG_edges.csv', sep ='\t', index=False)


In [ ]:
# interventions will have to be mapped to more than 1 vocabulary unlike conditions, which seems more easily mapped to SNOMED
# first look at the types of interventions (unique on int_type in ct_extract dataframe)

In [93]:
# adding Folder_2/subfolder to the system path
sys.path.insert(0, '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/pymetamap-master')
  
from pymetamap import MetaMap

In [99]:
# get the directory of the the module
print(os.getcwd())
# Path
path = "/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin"  
# Path of Start directory
start = os.getcwd()
print(os.path.relpath("/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin", os.getcwd()))

/Users/Kamileh/Work/ISB/NCATS_BiomedicalTranslator/Projects/ClinicalTrials/ETL_Python
../../../../../../../../Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin


In [102]:
mm = MetaMap.get_instance('/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin/metamap18')

In [175]:
sents = ['hypoandrogenism']  # this has to be 1 sentence
concepts,error = mm.extract_concepts(sents)
for concept in concepts:
    print(concept)
    print("\n")

ConceptMMI(index='USER', mm='MMI', score='3.68', preferred_name='Deficiency of testosterone biosynthesis', cui='C0342527', semtypes='[dsyn]', trigger='["Hypoandrogenism"-tx-1-"hypoandrogenism"-noun-0]', location='TX', pos_info='4/15', tree_codes='')




In [147]:
test1 = list((final_df['subject_name'][:20]))
for ct_string in test1:
#     print(ct_string)
    ct_string_list = [ct_string]
    print(ct_string_list)
    print(mm.extract_concepts(ct_string_list))
    


['autism spectrum disorder']
([ConceptMMI(index='USER', mm='MMI', score='13.27', preferred_name='Autism Spectrum Disorders', cui='C1510586', semtypes='[mobd]', trigger='["AUTISM SPECTRUM DISORDER"-tx-1-"autism spectrum disorder"-noun-0]', location='TX', pos_info='4/24', tree_codes='F03.625.164.113'), ConceptMMI(index='USER', mm='MMI', score='3.81', preferred_name='Pervasive Development Disorder', cui='C0524528', semtypes='[mobd]', trigger='["Autism spectrum disorder"-tx-1-"autism spectrum disorder"-noun-0]', location='TX', pos_info='4/24', tree_codes='')], None)
['autism spectrum disorder']
([ConceptMMI(index='USER', mm='MMI', score='13.27', preferred_name='Autism Spectrum Disorders', cui='C1510586', semtypes='[mobd]', trigger='["AUTISM SPECTRUM DISORDER"-tx-1-"autism spectrum disorder"-noun-0]', location='TX', pos_info='4/24', tree_codes='F03.625.164.113'), ConceptMMI(index='USER', mm='MMI', score='3.81', preferred_name='Pervasive Development Disorder', cui='C0524528', semtypes='[mobd

([ConceptMMI(index='USER', mm='MMI', score='22.61', preferred_name='Small Nuclear Ribonucleoprotein Polypeptide F', cui='C1367790', semtypes='[aapp,bacs]', trigger='["Sma"-tx-1-"sma"-noun-0]', location='TX', pos_info='4/3', tree_codes='D12.776.157.725.500.875.700;D12.776.660.625.750;D12.776.664.962.500.875.700;D23.050.290.875;D23.050.422.750'), ConceptMMI(index='USER', mm='MMI', score='16.30', preferred_name='insulin-like growth factor 2', cui='C0021666', semtypes='[aapp,bacs]', trigger='["SM-A"-tx-1-"sma"-noun-0]', location='TX', pos_info='4/3', tree_codes='D12.644.276.937.420;D12.776.124.862.425;D12.776.467.937.420;D23.529.937.420'), ConceptMMI(index='USER', mm='MMI', score='13.14', preferred_name='Spinal Muscular Atrophy', cui='C0026847', semtypes='[dsyn]', trigger='["SMA"-tx-1-"sma"-noun-0]', location='TX', pos_info='4/3', tree_codes='C10.228.854.468;C10.574.562.500;C10.668.467.500'), ConceptMMI(index='USER', mm='MMI', score='3.68', preferred_name='SMN1 wt Allele', cui='C4284041', 

In [171]:
# test metamap on dataframe now
# pre-process disease column
diseases = set(list(final_df['subject_name']))
processed_diseases = [''.join(str(disease).split(',')) for disease in diseases]
processed_diseases = [''.join(str(disease).split(';')) for disease in processed_diseases]
# processed_diseases

In [181]:
# disease_concepts = []

# for disease in processed_diseases[:10]:
#     disease = [disease]
#     concepts,error = mm.extract_concepts(disease)
# #     print(disease)
# #     print(concepts)
# #     print("\n")
#     disease_concepts.append(concepts)
    
    
    
# test6 = pd.DataFrame(zip(diseases, disease_concepts))    


In [ ]:
disease_concepts = [mm.extract_concepts([disease]) for disease in processed_diseases]

In [ ]:
disease_concepts